In [2]:
import pandas as pd
import numpy as np
import os
import subprocess
import datetime
from pathlib import Path

In [3]:
metadata = pd.read_csv("../data/metadata_2021_04_08.tsv", sep="\t", parse_dates = ["Collection date", "Submission date"])

metadata = metadata[["Accession ID", \
                   "Collection date", \
                   "Submission date", \
                   "Location", \
                   "Additional location information", \
                   "Sequence length", \
                   "Host", \
                   "AA Substitutions", \
                   "Is reference?"]]
def get_nth_slash(row, n):
    try:
        return row.split("/")[n].strip()
    except:
        return np.nan
    
for i in range(4):
    metadata[f"Location_{i}"] = metadata["Location"].apply(lambda row: get_nth_slash(row, i))
metadata = metadata.rename(columns={"Location_1":"country", "Location_2":"state"})

/apps/software/standard/compiler/gcc/9.2.0/jupyter_conda/2019.10-py3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (13,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
def prep_gengraph_input_output(date, end_date, seq_names):
    date = date.strftime("%Y-%m-%d")
    end_date = end_date.strftime("%Y-%m-%d")
    
    output_path = Path("/sfs/lustre/bahamut/scratch/jho5ze/bionets/covid/data/Houston_output")
    run_path = output_path / f"{date}_{end_date}"
    
    if not run_path.exists():
        run_path.mkdir()
        
    seq_path = Path("/sfs/lustre/bahamut/scratch/jho5ze/bionets/covid/data/Houston")
    seq_file_dest = run_path / f"{date}_{end_date}_seq_input.txt"
    with open(seq_file_dest, "w") as dest:
        dest.write("seq_name\taln_name\tseq_path\tannotation_path\n")
        for ix, seq in enumerate(seq_names):
            dest.write(f"{seq}\tseq_{ix}\t{seq_path / seq}.fasta\n")
            
    return run_path, seq_file_dest, f"{date}_{end_date}"

def run_pipeline(date, end_date, seq_names, exp_name, location="Houston"):
    location = location.replace(" ", "_")
    base_path = Path("/scratch/jho5ze/bionets/covid/")
    uncert_file = "uncert.csv"
    
    date = date.strftime("%Y-%m-%d")
    end_date = end_date.strftime("%Y-%m-%d")
    
    output_path = Path("/sfs/lustre/bahamut/scratch/jho5ze/bionets/covid/data/output")
    run_path = output_path / location / exp_name / f"{date}_{end_date}"
    uncert_file = run_path / "uncert.csv"
    
    
    if not run_path.exists():
        run_path.mkdir(parents=True)
        
    seq_path = Path("/sfs/lustre/bahamut/scratch/jho5ze/bionets/covid/data/Houston")
    
    command = f"sbatch {base_path / 'scripts/run_pipeline.sbatch'} {run_path} {uncert_file}".split()
#     command += [f"{seq_path / seq}.fasta" for seq in seq_names]
    command += [f"{seq}" for seq in seq_names]
    return command
#     subprocess.run(command)
#     with open(seq_file_dest, "w") as dest:
#         dest.write("seq_name\taln_name\tseq_path\tannotation_path\n")
#         for ix, seq in enumerate(seq_names):
#             dest.write(f"{seq}\tseq_{ix}\t{seq_path / seq}.fasta\n")
            
#     return run_path, seq_file_dest, f"{date}_{end_date}"

In [14]:
timeout = """2020-07-06_2020-07-10
2021-01-02_2021-01-06
2021-01-12_2021-01-16
2021-01-17_2021-01-21""".split()

In [14]:
seqs = 1
genomes = 1
if not (seqs ^ genomes): 
    print("yep")

yep


In [ ]:
data_path = Path("/scratch/jho5ze/bionets/covid/data")

seqs = pd.read_csv(data_path / "houston_metadata.csv", header=None, parse_dates=[1])
seqs.columns = ["seq", "date"]

date_window = 7
date_step_size = 7

for date in pd.date_range(seqs.date.min(), seqs.date.max(), freq=f"{date_step_size}D"):
    end_date = date + np.timedelta64(date_window - 1, 'D')
    sub_seqs = seqs[(seqs.date >= date) & (seqs.date <= end_date)].seq
#     output_directory, seq_file, out_file = prep_gengraph_input_output(date, end_date, sub_seqs)
    experiment_name = f"window_{date_window}_step_{date_step_size}"
    stdate = date.strftime("%Y-%m-%d")
    stend_date = end_date.strftime("%Y-%m-%d")
    
#     if f"{stdate}_{stend_date}" in timeout:
#     command = run_pipeline(date, end_date, sub_seqs, experiment_name)
#     subprocess.run(command)
#     print(f"{stdate}_{stend_date}")
#     print("\t", len(sub_seqs))
#     break
#     print(" ".join(command))
#     break
        
#         command = f"sbatch {base_path / 'scripts/run_gengraph.sbatch'} {output_directory} {seq_file} {out_file}"
#         subprocess.run(command.split())
#     break


In [16]:
location = "Houston"
location_metadata = metadata[metadata.Location_3 == location]
location_metadata = location_metadata[["Accession ID", "Collection date"]]
location_metadata.columns = ["seq", "date"]

date_window = 7
date_step_size = 7

for date in pd.date_range(location_metadata.date.min(), location_metadata.date.max(), freq=f"{date_step_size}D"):
    end_date = date + np.timedelta64(date_window - 1, 'D')
    sub_seqs = location_metadata[(location_metadata.date >= date) & (location_metadata.date <= end_date)].seq
#     output_directory, seq_file, out_file = prep_gengraph_input_output(date, end_date, sub_seqs)
    experiment_name = f"window_{date_window}_step_{date_step_size}"
    stdate = date.strftime("%Y-%m-%d")
    stend_date = end_date.strftime("%Y-%m-%d")
    print(stdate + " " + stend_date)
    command = run_pipeline(date, end_date, sub_seqs, experiment_name)
    
    print(command)
    brea

2020-03-18 2020-03-24
['sbatch', '/scratch/jho5ze/bionets/covid/scripts/run_pipeline.sbatch', '/sfs/lustre/bahamut/scratch/jho5ze/bionets/covid/data/output/Houston/window_7_step_7/2020-03-18_2020-03-24', '/sfs/lustre/bahamut/scratch/jho5ze/bionets/covid/data/output/Houston/window_7_step_7/2020-03-18_2020-03-24/uncert.csv', 'EPI_ISL_545707', 'EPI_ISL_545714']
2020-03-25 2020-03-31
['sbatch', '/scratch/jho5ze/bionets/covid/scripts/run_pipeline.sbatch', '/sfs/lustre/bahamut/scratch/jho5ze/bionets/covid/data/output/Houston/window_7_step_7/2020-03-25_2020-03-31', '/sfs/lustre/bahamut/scratch/jho5ze/bionets/covid/data/output/Houston/window_7_step_7/2020-03-25_2020-03-31/uncert.csv', 'EPI_ISL_542578', 'EPI_ISL_542581', 'EPI_ISL_542585', 'EPI_ISL_542587', 'EPI_ISL_542590', 'EPI_ISL_542610', 'EPI_ISL_542615', 'EPI_ISL_542617', 'EPI_ISL_542795', 'EPI_ISL_542803', 'EPI_ISL_542805', 'EPI_ISL_542821', 'EPI_ISL_543980']
2020-04-01 2020-04-07
['sbatch', '/scratch/jho5ze/bionets/covid/scripts/run_pipe